Structure URL d'une recherche Marmiton : "https://www.marmiton.org/recettes/recherche.aspx?aqt=" + "nom-du-plat"

Importation des modules nécessaires au Webscraping

Test de récupération d'un code source avec la recherche "Steak tartare"

In [19]:
import requests
from bs4 import BeautifulSoup
import lxml

In [20]:
#Transformation de la recherche en texte compatible URL, minuscules et tirets entre les mots
def to_url(search):
    return (search.lower()).replace(' ', '-')

s = 'Steak tartare'

url_st = "https://www.marmiton.org/recettes/recherche.aspx?aqt=" + to_url(s)

requests.get(url_st).status_code

200

La page de recherche ainsi renseignée possède bien un contenu : c'est déjà ça.

In [24]:
response = requests.get(url_st).text
st = BeautifulSoup(response)

In [25]:
import re
import numpy as np

In [34]:
found_recipes = st.findAll('a', {'class' : 'MRTN__sc-1gofnyi-2 gACiYG'})
print(len(found_recipes))

12


L'attribut "class = MRTN__sc-1gofnyi-2 gACiYG" semble se retrouver pour toutes les pages de résultats. Il semble être ce qui caractérise les résultats.

Attention : le site de Marmiton n'affiche que 12 résultats par page. Pour la recherche steak tartare, le site annonce un total de 16 résultats mais la commande ci-dessus ne prend que les 12 premiers, puisqu'il ne traite que la première page de résultats. Il faudra trouver un moyen de traiter TOUTES les pages.

In [37]:
url_results = []
for recipe in found_recipes:
    href = recipe.get('href')
    url_results.append('https://www.marmiton.org' + href)
print(len(url_results))
print(url_results)

12
['https://www.marmiton.org/recettes/recette_steak-tartare_18121.aspx', 'https://www.marmiton.org/recettes/recette_steak-tartare-arrange_50144.aspx', 'https://www.marmiton.org/recettes/recette_steak-tartare-vegetarien_30154.aspx', 'https://www.marmiton.org/recettes/recette_sauce-legere-et-relevee-pour-steack-tartare_222507.aspx', 'https://www.marmiton.org/recettes/recette_steak-tartare-a-l-italienne-tomate-sechee-et-mozzarella_315226.aspx', 'https://www.marmiton.org/recettes/recette_steak-tartare-maison_22175.aspx', 'https://www.marmiton.org/recettes/recette_steak-tartare-a-la-coreenne_43082.aspx', 'https://www.marmiton.org/recettes/recette_steak-tartare-au-poivron-rouge_28172.aspx', 'https://www.marmiton.org/recettes/recette_steak-tartare-au-vinaigre-balsamique_73859.aspx', 'https://www.marmiton.org/recettes/recette_steak-tartare-et-riz-aux-legumes_28609.aspx', 'https://www.marmiton.org/recettes/recette_steak-tartare-a-l-americaine_27808.aspx', 'https://www.marmiton.org/videos/steak

Tentons maintenant de récupérer les ingrédients d'une recette.

In [38]:
url_test = url_results[0] #On prend pour exemple le premier résultat de la recherche "steak tartare"
requests.get(url_test).status_code

200

In [39]:
recipe = requests.get(url_test).text
recipeST = BeautifulSoup(recipe)

In [48]:
ingredients_table = recipeST.find('div', {'class' : 'MuiGrid-root RCP__sc-vgpd2s-6 ghZzUe MuiGrid-container MuiGrid-spacing-xs-2'})
ingredients1 = ingredients_table.findAll('span', {'class' : 'RCP__sc-8cqrvd-3 itCXhd'})
ingredients2 = ingredients_table.findAll('span', {'class' : 'RCP__sc-8cqrvd-3 cDbUWZ'})
ingredients = ingredients1 + ingredients2
noms_ingredients = []
for ingredient in ingredients:
    nom_ingredient = ingredient.string
    noms_ingredients.append(nom_ingredient)
print(noms_ingredients)

['moutarde de Dijon', 'oignon', 'câpres', 'sauce worcestershire', 'ketchup', 'tabasco', 'poivre', 'sel', "huile d'olive", 'persil', 'boeuf', "jaune d'oeuf"]


On a récupéré la liste des ingrédients de la recette du steak tartare ! Essayons maintenant de choper les quantités.

In [54]:
qte = ingredients_table.findAll('span', {'class' : 'SHRD__sc-10plygc-0 epviYI'})
qtes = []
for q in qte:
    print(q.text)

1 c.à.c
1 c.à.s
1 c.à.s
1 c.à.c
1 c.à.s
 


2 c.à.s
1 c.à.c
250 g
1


On récupère en effet la quantité. Toutefois il faudra gérer le cas où la quantité n'est pas spécifiée (comme par exemple "sel").